In [4]:
import numpy as np
import pandas as pd
import patsy

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.cross_validation import train_test_split, StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.grid_search import GridSearchCV

In [5]:
# prep data, covert date to datetime, split for train and test, and build model
sf_crime = pd.read_csv('../../assets/datasets/sf_crime_train.csv')

sf_crime = sf_crime.dropna()

sf_crime['Dates'] = pd.to_datetime(sf_crime.Dates)
sf_crime_dates = pd.DatetimeIndex(sf_crime.Dates.values, dtype='datetime64[ns]', freq=None)

sf_crime['hour'] = sf_crime_dates.hour
sf_crime['month'] = sf_crime_dates.month
sf_crime['year'] = sf_crime_dates.year

In [6]:
subset = ['VEHICLE THEFT','BURGLARY','DRUG/NARCOTIC']
sf_crime_sub = sf_crime[sf_crime['Category'].str.contains('|'.join(subset))]

#sf_sample = sf_crime_sub.sample(frac=0.50)

X = patsy.dmatrix('~ C(hour) + C(DayOfWeek) + C(PdDistrict)', sf_crime_sub)
Y = sf_crime_sub.Category.values

# split for train and test
sss = StratifiedShuffleSplit(Y, 1, test_size=0.33, random_state=0)
print len(sss)

for train_index, test_index in sss:
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]


1


In [7]:
# fit model with five folds and lasso regularization
# use Cs=15 to test a grid of 15 distinct parameters
# remeber: Cs describes the inverse of regularization strength
logreg_cv = LogisticRegressionCV(Cs=15, cv=5, penalty='l1', scoring='accuracy', solver='liblinear')
logreg_cv.fit(X_train, Y_train)

LogisticRegressionCV(Cs=15, class_weight=None, cv=5, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=1, penalty='l1', refit=True,
           scoring='accuracy', solver='liblinear', tol=0.0001, verbose=0)

In [8]:
# find best C per class
print('best C for class:')
best_C = {logreg_cv.classes_[i]:x for i, (x, c) in enumerate(zip(logreg_cv.C_, logreg_cv.classes_))}
print(best_C)

best C for class:
{'BURGLARY': 193.06977288832496, 'VEHICLE THEFT': 3.7275937203149381, 'DRUG/NARCOTIC': 1.0}


In [9]:
# fit regular logit model to 'DRUG/NARCOTIC' and 'BURGLARY' classes
# use lasso penalty
logreg_1 = LogisticRegression(C=best_C['DRUG/NARCOTIC'], penalty='l1', solver='liblinear')
logreg_2 = LogisticRegression(C=best_C['BURGLARY'], penalty='l1', solver='liblinear')

logreg_1.fit(X_train, Y_train)
logreg_2.fit(X_train, Y_train)

LogisticRegression(C=193.06977288832496, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', penalty='l1', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0)

In [10]:
# build confusion matrices for the models above
Y_1_pred = logreg_1.predict(X_train)
Y_2_pred = logreg_2.predict(X_train)

conmat_1 = confusion_matrix(Y_train, Y_1_pred, labels=logreg_1.classes_)
conmat_1 = pd.DataFrame(conmat_1, columns=logreg_1.classes_, index=logreg_1.classes_)

conmat_2 = confusion_matrix(Y_train, Y_2_pred, labels=logreg_2.classes_)
conmat_2 = pd.DataFrame(conmat_2, columns=logreg_2.classes_, index=logreg_2.classes_)

print(conmat_1)
print(conmat_2)

               BURGLARY  DRUG/NARCOTIC  VEHICLE THEFT
BURGLARY            208             18            264
DRUG/NARCOTIC        79             94            159
VEHICLE THEFT       124             16            508
               BURGLARY  DRUG/NARCOTIC  VEHICLE THEFT
BURGLARY            215             26            249
DRUG/NARCOTIC        69            120            143
VEHICLE THEFT       125             26            497


In [11]:
# print classification reports
print(classification_report(Y_train, Y_1_pred))
print(classification_report(Y_train, Y_2_pred))

             precision    recall  f1-score   support

   BURGLARY       0.51      0.42      0.46       490
DRUG/NARCOTIC       0.73      0.28      0.41       332
VEHICLE THEFT       0.55      0.78      0.64       648

avg / total       0.58      0.55      0.53      1470

             precision    recall  f1-score   support

   BURGLARY       0.53      0.44      0.48       490
DRUG/NARCOTIC       0.70      0.36      0.48       332
VEHICLE THEFT       0.56      0.77      0.65       648

avg / total       0.58      0.57      0.55      1470



In [43]:
# run gridsearch using GridSearchCV and 5 folds
# score on f1_macro; what does this metric tell us?
logreg = LogisticRegression()
C_vals = [0.0001, 0.001, 0.01, 0.1, 0.5, 0.75, 1.0, 2.5, 5.0, 10.0, 100.0, 1000.0]
penalties = ['l1','l2']

gs = GridSearchCV(logreg, {'penalty':penalties, 'C':C_vals}, verbose=True, cv=5, scoring='f1_macro')
gs.fit(X, Y)

[Parallel(n_jobs=1)]: Done  49 tasks       | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:    1.8s finished


Fitting 5 folds for each of 24 candidates, totalling 120 fits


GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'penalty': ['l1', 'l2'], 'C': [0.0001, 0.001, 0.01, 0.1, 0.5, 0.75, 1.0, 2.5, 5.0, 10.0, 100.0, 1000.0]},
       pre_dispatch='2*n_jobs', refit=True, scoring='f1_macro',
       verbose=True)

In [44]:
# find the best parameter
gs.best_params_

{'C': 10.0, 'penalty': 'l1'}

In [45]:
# use this parameter to .fit, .predict, and print a classification_report for our X and Y
gs_logreg = LogisticRegression(C=10, penalty='l1', solver='liblinear')
gs_logreg.fit(X, Y)
Y_ = gs_logreg.predict(X)
print(classification_report(Y, Y_))

             precision    recall  f1-score   support

   BURGLARY       0.51      0.43      0.47       732
DRUG/NARCOTIC       0.73      0.31      0.43       496
VEHICLE THEFT       0.56      0.79      0.65       967

avg / total       0.58      0.56      0.54      2195

